1. Définir la question à répondre

1.1 Combien de municipalités sont à risque de feu de forêt au Québec ?
    1.1.1 Quelle est la zone de feu de forêt au Québec ?
    1.1.2 Combien il y a de municipalité inclus dans cette zone ?

1.2 Identifier les biens et les infrastructures les plus exposés aux feux de forêt à l’échelle des communautés.

2. Définir les sources de données pour cette analyse.
    2.1 Identifier la zone de couverture du jeu de données
    2.2 Identifier le CRS

In [2]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
import requests, zipfile, io

# Defini variables
rawDataRelativePath = './rawData'


In [3]:
# 2. Définir les sources de données pour cette analyse

# Municipalités
regionalUrl = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Regional/SDA_100k/SHP/BDAT(adm)_SHP.zip'
# Aéroport
airportsUrl = 'https://ws.mapserver.transports.gouv.qc.ca/swtq?service=wfs&version=2.0.0&request=getfeature&typename=ms:evenements&outfile=Aeroport&srsname=EPSG:4326&outputformat=geojson'
# villages autochtones
indigenousVillagesUrl = 'https://diffusion.mern.gouv.qc.ca/Diffusion/RGQ/Documentation/BDVA/Index_BDVA.zip'

firesUrl = 'https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Foret/PERTURBATIONS_NATURELLES/Feux_foret/FEUX_PROV_GPKG.zip'

# Empreintes de bâtiments
buildingFootPrint = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Local/Batiments/FGDB/QC_600019-54_Prevention_risques_sinistres_2019_fgdb.zip'

In [4]:
# Downloads the data sources in geojson format
responseBody = requests.get(airportsUrl, stream=True).json()
airports = gpd.GeoDataFrame.from_features(responseBody['features'], crs="EPSG:4326")
# airports.head(10)

In [ ]:
# Downloads the data sources in .zip format
# Loading all raw data can take a moment
response = requests.get(firesUrl)
zipFile = zipfile.ZipFile(io.BytesIO(response.content))
zipFile.extractall(path=rawDataRelativePath)

In [ ]:
targetFile = 'FEUX_PROV.gpkg'
pathAndFile = "%s/%s" % (rawDataRelativePath, targetFile)
provincialFiresGeom = gpd.read_file(pathAndFile)
# provincialFiresGeom.head()
provincialFiresGeom.crs

# Use Case from .zip file
response = requests.get(regionalUrl)
zipFile = zipfile.ZipFile(io.BytesIO(response.content))
zipFile.extractall(path=rawDataRelativePath)

In [ ]:
targetFile = 'regio_s.shp'
pathAndFile = "%s/Bdat/SHP/%s" % (rawDataRelativePath, targetFile)
regions = gpd.read_file(pathAndFile)
# regions.explore()
regions.crs

In [12]:
# Create a DataFrame that represent the entire province
quebecProvince = regions.dissolve(by='RES_DE_IND')

In [15]:
# Create a custom rectangular to clip the quebec under 52 latitude
# source: https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
rectangular = pd.DataFrame(
    {
        "Country": ["Canada"],
        "Coordinates": [
            "POLYGON ((-80 52, -55 52, -55 45, -80 45, -80 52))"
        ],
    }
)
rectangular["Coordinates"] = gpd.GeoSeries.from_wkt(rectangular["Coordinates"])
rectangular = gpd.GeoDataFrame(rectangular, geometry="Coordinates", crs="EPSG:4269")
clippedQuebecUnder52Latitude = quebecProvince.clip(rectangular)
clippedQuebecUnder52Latitude.explore()
